<a href="https://colab.research.google.com/github/MayankKhoria2007/CSESA-PS5-SUMMARIZER/blob/main/SUMMARIZERT5HYBRIDWITHTEXTRANK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"]="1"

In [ ]:
import nltk
import torch
import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from datasets import load_dataset
ds=load_dataset("NortheasternUniversity/big_patent","a",split="train[:6%]")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

a/train-00000-of-00012.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

a/train-00001-of-00012.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

a/train-00002-of-00012.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

...

a/validation-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

a/test-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/174134 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9674 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9675 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
model_name="t5-small"
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix="summarize"
def preprocess(examples):
  input=[prefix+ i for i in examples["description"]]

  model_inputs=tokenizer(input,max_length=512,truncation=True,padding="max_length")
  labels=tokenizer(text_target=examples["abstract"],max_length=128,truncation=True,padding="max_length")
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs
raw_dataset=ds.train_test_split(test_size=0.2)
eval_dataset=raw_dataset["test"]
toeknized_ds=ds.map(preprocess,batched=True,remove_columns=ds.column_names)

Map:   0%|          | 0/10448 [00:00<?, ? examples/s]

In [ ]:

toeknized_ds=toeknized_ds.train_test_split(test_size=0.2)


In [ ]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d259b4e56229a19c32f2cdcc673398e3112e6432400b575aa8c18e8e0bbba0d6
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import evaluate
import numpy as np
rouge=evaluate.load("rouge")
def compute_metrics(eval_pred):
  predictions,labels=eval_pred
  decoded_preds=tokenizer.batch_decode(predictions,skip_special_tokens=True)
  labels=np.where(labels!=-100,labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.batch_decode(labels,skip_special_tokens=True)
  result=rouge.compute(predictions=decoded_preds,references=decoded_labels,use_stemmer=True)
  prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"]=np.mean(prediction_lens)
  return {k:round(v,4) for k,v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model_name,padding="longest")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-bigpatent",
    eval_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.01,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=False,
    remove_unused_columns=False,
    report_to="none",
    fp16=False)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=toeknized_ds["train"],
    eval_dataset=toeknized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.357600,2.176394
2,2.311400,2.131923
3,2.257100,2.110403


Epoch,Training Loss,Validation Loss
1,2.357600,2.176394
2,2.311400,2.131923
3,2.257100,2.110403
4,2.205600,2.103671


TrainOutput(global_step=4180, training_loss=2.301776133998159, metrics={'train_runtime': 2014.1214, 'train_samples_per_second': 16.599, 'train_steps_per_second': 2.075, 'total_flos': 4524747106811904.0, 'train_loss': 2.301776133998159, 'epoch': 4.0})

In [ ]:
!pip install bert_score
bert=evaluate.load("bertscore")

In [ ]:
model.save_pretrained("./t5_bigpatent_model")
tokenizer.save_pretrained("./t5_bigpatent_model")


('./t5_bigpatent_model/tokenizer_config.json',
 './t5_bigpatent_model/special_tokens_map.json',
 './t5_bigpatent_model/spiece.model',
 './t5_bigpatent_model/added_tokens.json',
 './t5_bigpatent_model/tokenizer.json')

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def lemmatize_sentence(sentence):
    words = word_tokenize(sentence.lower())
    words = [
        lemmatizer.lemmatize(w)
        for w in words
        if w.isalpha() and w not in stop_words
    ]
    return " ".join(words)


In [ ]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
def textrank_extractive_lemmatized(text, k=7):
    sentences = sent_tokenize(text)
    processed = [lemmatize_sentence(s) for s in sentences]
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf = vectorizer.fit_transform(processed)
    similarity_matrix = cosine_similarity(tfidf)
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(
        ((scores[i], s, i) for i, s in enumerate(sentences)),
        reverse=True
    )
    selected = sorted(ranked_sentences[:k], key=lambda x: x[2])
    extracted_text = " ".join([s[1] for s in selected])
    return extracted_text




In [ ]:

from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="./t5_bigpatent_model",   # your trained model path
    tokenizer="./t5_bigpatent_model",
    device=0 if torch.cuda.is_available() else -1
)

def hybrid_summarizer_pipeline(text):
    # Extractive phase
    extracted_text = textrank_extractive_lemmatized(text, k=7)
    extracted_text=extracted_text[:1500]
    torch.cuda.empty_cache()
    # Abstractive phase using pipeline
    with torch.no_grad():
      summary = summarizer(
          "summarize:"+extracted_text,
          max_length=150,
          min_length=40,
          num_beams=2,
          do_sample=False
      )
    torch.cuda.empty_cache()
    return summary[0]["summary_text"]



predictions = []
references = []
c=0

for ex in eval_dataset.select(range(200)):
    text = ex["description"]
    ref = ex["abstract"]

    pred = hybrid_summarizer_pipeline(text)


    predictions.append(pred)
    references.append(ref)



Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 150, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_c

In [ ]:
P,R,F1 = bert.compute(predictions=predictions, references=references,lang='en')
print(f"BERTScore Precision:{P.mean().item():.4f}")
print(f"BERTScore Recall:{R.mean().item():.4f}")
print(f"BERTScore F1:{F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [-0.009386022575199604, -0.013384093530476093, -0.06172991171479225, -0.16346953809261322, 0.04260093346238136, 0.19250774383544922, 0.039021700620651245, 0.5605206489562988, 0.4170626699924469, 0.29530224204063416, 0.3072897493839264, 0.08155860751867294, 0.33004996180534363, 0.04077452793717384, 0.14525748789310455, 0.021762285381555557, -0.039746884256601334, 0.03863894194364548, -0.10092706978321075, 0.0276122335344553, -0.034639809280633926, 0.18340224027633667, 0.36091843247413635, 0.2859116196632385, 0.17314457893371582, 0.18425123393535614, 0.1871710866689682, 0.17411774396896362, 0.19959120452404022, -0.031831398606300354, 0.06639479845762253, 0.0455947108566761, 0.06745533645153046, 0.2584051787853241, 0.2508264482021332, 0.29611092805862427, 0.432991623878479, 0.3101080656051636, -0.01619710586965084, 0.2198801338672638, 0.10761044919490814, 0.06757596880197525, 0.22071391344070435, 0.3834494352340698, 0.3911070227622986, 0.22169768810272217, -0.032431002706289

In [ ]:
t="Artificial intelligence systems are increasingly being integrated into modern technological solutions across multiple industries. These systems rely on advanced machine learning algorithms that enable computers to learn patterns from large volumes of data and make intelligent decisions. In recent years, deep learning techniques based on neural networks have shown significant improvements in tasks such as image recognition, natural language processing, and speech analysis. However, training deep learning models requires substantial computational resources, including high-performance processors and graphical processing units. The growing size of datasets and model parameters has made scalability a major challenge. To address this issue, researchers have proposed optimization techniques such as model compression, distributed training, and hybrid processing pipelines. Hybrid systems combine multiple approaches to improve efficiency and performance. For example, in text summarization, extractive methods can be used to identify the most relevant portions of a document, while abstractive models generate concise and fluent summaries. This combination reduces input length while preserving essential information. Such hybrid approaches are particularly useful for processing long technical documents, including patents and research articles. By integrating statistical methods with neural models, hybrid systems achieve better accuracy, faster processing, and improved adaptability to real-world applications."
print(hybrid_summarizer_pipeline(t))

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


A deep learning system based on neural networks has improved image recognition, natural language processing, and speech analysis. The system includes a high-performance processor, distributed training, and hybrid processing pipelines. The hybrid systems combine multiple approaches to improve efficiency and performance. This combination reduces input length while preserving essential information.
